In [1]:
# import os

# os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

tf.__version__

'2.6.0'

In [3]:
# !tf2/bin/pip3 install malaya-speech --no-deps
# !tf2/bin/pip3 install python-speech-features tqdm unidecode malaya-boilerplate librosa herpetologist

In [4]:
import malaya_speech
import numpy as np

/home/husein/tf2/lib/python3.6/site-packages/malaya_boilerplate/frozen_graph.py:29: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  'Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0'


In [5]:
m = hub.KerasLayer('https://tfhub.dev/google/nonsemantic-speech-benchmark/trillsson2/1',
                  trainable=False)

In [6]:
import json
import pickle

with open('/home/husein/youtube/voxceleb2-test-sample.json') as fopen:
    sample_files = json.load(fopen)

In [7]:
with open('/home/husein/youtube/voxceleb2-test-labels.pkl', 'rb') as fopen:
    labels = pickle.load(fopen)

In [8]:
y, _ = malaya_speech.load(sample_files[0])

In [10]:
%%time

m(np.expand_dims(y, 0))['embedding'].numpy()[0]

CPU times: user 595 ms, sys: 170 ms, total: 765 ms
Wall time: 75.7 ms


array([-0.8584541 ,  0.19205517, -0.20843373, ..., -0.8996028 ,
        0.52223045, -0.23444548], dtype=float32)

In [11]:
unique_files = []
for l in labels:
    unique_files.extend(l[1:])
    
unique_files = list(set(unique_files))  

In [12]:
from tqdm import tqdm

vectors = {}
for f in tqdm(unique_files):
    y, _ = malaya_speech.load(f)
    v = m(np.expand_dims(y, 0))['embedding'].numpy()[0]
    vectors[f] = v

100%|██████████| 36237/36237 [2:04:38<00:00,  4.85it/s]   


In [13]:
import numpy as np

scores, ls = [], []

for i in tqdm(range(len(labels))):
    ls.append(labels[i][0])
    scores.append(np.sum(vectors[labels[i][1]] * vectors[labels[i][2]]))

100%|██████████| 5900000/5900000 [01:46<00:00, 55527.38it/s] 


In [14]:
def calculate_eer(y, y_score):
    
    from scipy.optimize import brentq
    from sklearn.metrics import roc_curve
    from scipy.interpolate import interp1d

    fpr, tpr, thresholds = roc_curve(y, y_score, pos_label=1)
    eer = brentq(lambda x : 1. - x - interp1d(fpr, tpr)(x), 0., 1.)
    thresh = interp1d(fpr, thresholds)(eer)
    return eer, thresh

In [15]:
calculate_eer(ls, scores)

(0.38987999999991463, array(9207.43115234))